In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn
from scipy.stats import poisson,skellam

from league_team_names import epl_team_dict, bundesliga_team_dict, laliga_team_dict, seriea_teams_dict

country_group_dict = pd.read_csv('resources/worldcup_country_groups.csv', ).to_dict

In [67]:
players_df = pd.read_csv('resources/fifa-18-demo-player-dataset/CompleteDataset.csv')
players_df.drop(['Unnamed: 0', 'Flag', 'Club Logo', 'Value', 'Wage', 'Photo', 'Potential', "Special"], axis=1, inplace=True)

players_df[['Name', 'Overall', 'Preferred Positions']].head()

/Users/Michael/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Name,Overall,Preferred Positions
0,Cristiano Ronaldo,94,ST LW
1,L. Messi,93,RW
2,Neymar,92,LW
3,L. Suárez,92,ST
4,M. Neuer,92,GK


In [68]:
players_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17981 entries, 0 to 17980
Data columns (total 67 columns):
Name                   17981 non-null object
Age                    17981 non-null int64
Nationality            17981 non-null object
Overall                17981 non-null int64
Club                   17733 non-null object
Acceleration           17981 non-null object
Aggression             17981 non-null object
Agility                17981 non-null object
Balance                17981 non-null object
Ball control           17981 non-null object
Composure              17981 non-null object
Crossing               17981 non-null object
Curve                  17981 non-null object
Dribbling              17981 non-null object
Finishing              17981 non-null object
Free kick accuracy     17981 non-null object
GK diving              17981 non-null object
GK handling            17981 non-null object
GK kicking             17981 non-null object
GK positioning         17981 non-null 

In [73]:
# positions_list = list(players_df['Preferred Positions'].apply(lambda x: x.split(' ', 1)[0]).drop_duplicates())
# positions_list

In [74]:
positions_dict = {'ST': 'Attack',
                  'RW': 'Attack',
                  'LW': 'Attack',
                  'GK': 'Defense',
                  'CDM': 'Midfield',
                  'CB': 'Defense',
                  'RM': 'Midfield',
                  'CM': 'Midfield',
                  'LM': 'Midfield',
                  'LB': 'Defense',
                  'CAM': 'Attack',
                  'RB': 'Defense',
                  'CF': 'Attack',
                  'RWB': 'Defense',
                  'LWB': 'Defense'}

In [75]:
overall_score_df = players_df[['Name', 'Club', 'Nationality', 'Preferred Positions', 'Overall']].copy()
overall_score_df['Preferred Positions'] = overall_score_df['Preferred Positions'].apply(lambda x: x.split(' ', 1)[0]).replace(positions_dict)

team = 'Brazil'
overall_score_df[overall_score_df['Nationality'] == team].head(14).groupby(['Nationality', 'Preferred Positions']).mean()


Overall
Nationality Preferred Positions           
Brazil      Attack               87.333333
            Defense              85.250000
            Midfield             84.666667

In [76]:
epl2018 = pd.read_csv('resources/leagues/epl2018.csv')
bundesliga2018 = pd.read_csv('resources/leagues/bundesliga2018.csv')
seriea2018 = pd.read_csv('resources/leagues/seriea2018.csv')
laliga2018 = pd.read_csv('resources/leagues/laliga2018.csv')

epl2018.head()

,Unnamed: 0,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,0,E0,11/08/17,Arsenal,Leicester,4,3,H,2,2,...,2.32,21,-1.00,1.91,1.85,2.10,2.02,1.49,4.73,7.25
1,1,E0,12/08/17,Brighton,Man City,0,2,A,0,0,...,2.27,20,1.50,1.95,1.91,2.01,1.96,11.75,6.15,1.29
2,2,E0,12/08/17,Chelsea,Burnley,2,3,A,0,3,...,2.23,20,-1.75,2.03,1.97,1.95,1.90,1.33,5.40,12.25
3,3,E0,12/08/17,Crystal Palace,Huddersfield,0,3,A,0,2,...,1.72,18,-0.75,2.10,2.05,1.86,1.83,1.79,3.56,5.51
4,4,E0,12/08/17,Everton,Stoke,1,0,H,1,0,...,1.76,19,-0.75,1.94,1.90,2.01,1.98,1.82,3.49,5.42


In [8]:
results_dict = {'H': 2, "D": 1, "A": 0}
cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
test_df = epl2018[cols].rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals', 'FTR': 'Results'}).copy()
test_df['Results'] = test_df['Results'].replace(results_dict)
test_df

,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Results
0,Arsenal,Leicester,4,3,2
1,Brighton,Man City,0,2,0
2,Chelsea,Burnley,2,3,0
3,Crystal Palace,Huddersfield,0,3,0
4,Everton,Stoke,1,0,2
5,Southampton,Swansea,0,0,1
6,Watford,Liverpool,3,3,1
7,West Brom,Bournemouth,1,0,2
8,Man United,West Ham,4,0,2
9,Newcastle,Tottenham,0,2,0


In [9]:
player_overall_df = players_df[['Name', 'Club', 'Nationality', 'Preferred Positions', 'Overall']].copy()
player_overall_df['Preferred Positions'] = player_overall_df['Preferred Positions'].apply(lambda x: x.split(' ', 1)[0]).replace(positions_dict)

In [10]:
def get_team_stats(team, type='Club', no_of_players=13, return_df=False):
    mask = overall_score_df[type] == team
    groupby_cols = [type, 'Preferred Positions']
    team_stats_df = player_overall_df[mask].head(no_of_players).groupby(groupby_cols).mean().round(0)
    if return_df == True:
        return team_stats_df
    else:
        df = team_stats_df.reset_index().pivot(type, 'Preferred Positions')
        attack = df['Overall'].values[0][0]
        mid = df['Overall'].values[0][1]
        defense = df['Overall'].values[0][2]
        return [attack, mid, defense]

In [11]:
get_team_stats('Brazil', type='Nationality', return_df=False)

[87.0, 86.0, 85.0]

In [12]:
get_team_stats('Chelsea')

[86.0, 85.0, 84.0]

In [13]:
team_cols = ['HomeTeam', 'AwayTeam']
epl2018[team_cols] = epl2018[team_cols].replace(epl_team_dict)
bundesliga2018[team_cols] = bundesliga2018[team_cols].replace(bundesliga_team_dict)
seriea2018[team_cols] = seriea2018[team_cols].replace(seriea_teams_dict)
laliga2018[team_cols] = laliga2018[team_cols].replace(laliga_team_dict)

In [14]:
# seriea2018['HomeTeam'].drop_duplicates().sort_values()

2             Atalanta
10    Benevento Calcio
3              Bologna
24            Cagliari
13       Chievo Verona
4              Crotone
18      Ferrara (SPAL)
15          Fiorentina
11               Genoa
1        Hellas Verona
5                Inter
0             Juventus
6                Lazio
16               Milan
17              Napoli
12                Roma
7            Sampdoria
8             Sassuolo
19              Torino
9              Udinese
Name: HomeTeam, dtype: object

In [15]:
results_dict = {'H': 'HomeWin', "D": 'Draw', "A": 'AwayWin'}
cols = ['HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']

league_results_df = pd.concat([epl2018, bundesliga2018, seriea2018, laliga2018])[cols]
league_results_df.rename(columns={'FTHG': 'HomeGoals', 'FTAG': 'AwayGoals', 'FTR': 'Result'}, inplace=True)
league_results_df['Result'] = league_results_df['Result'].replace(results_dict)
league_results_df['Goal diff'] = league_results_df['HomeGoals'] - league_results_df['AwayGoals']

league_results_df.reset_index(drop=True, inplace=True)
league_results_df

,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Result,Goal diff
0,Arsenal,Leicester City,4,3,HomeWin,1
1,Brighton & Hove Albion,Manchester City,0,2,AwayWin,-2
2,Chelsea,Burnley,2,3,AwayWin,-1
3,Crystal Palace,Huddersfield Town,0,3,AwayWin,-3
4,Everton,Stoke City,1,0,HomeWin,1
5,Southampton,Swansea City,0,0,Draw,0
6,Watford,Liverpool,3,3,Draw,0
7,West Bromwich Albion,Bournemouth,1,0,HomeWin,1
8,Manchester United,West Ham United,4,0,HomeWin,4
9,Newcastle United,Tottenham Hotspur,0,2,AwayWin,-2


In [16]:
# Chart this

In [17]:
league_results_df['Result'].value_counts()

HomeWin    655
AwayWin    440
Draw       351
Name: Result, dtype: int64

In [18]:
homeTeam_attack = []
homeTeam_mid = []
homeTeam_def = []
awayTeam_attack = []
awayTeam_mid = []
awayTeam_def = []
data = []


for index, row in league_results_df.iterrows():
    try:
        home_team_stats = get_team_stats(row['HomeTeam'])
        home_attack = home_team_stats[0]
        home_midfield = home_team_stats[1]
        home_defense = home_team_stats[2]

        away_team_stats = get_team_stats(row['AwayTeam'])
        away_attack = away_team_stats[0]
        away_midfield = away_team_stats[1]
        away_defense = away_team_stats[2]
                
        homeTeam_attack.append(home_attack)
        homeTeam_mid.append(home_midfield)
        homeTeam_def.append(home_defense)
        awayTeam_attack.append(away_attack)
        awayTeam_mid.append(away_midfield)
        awayTeam_def.append(away_defense)

        
    except IndexError:
        homeTeam_attack.append(None)
        homeTeam_mid.append(None)
        homeTeam_def.append(None)
        awayTeam_attack.append(None)
        awayTeam_mid.append(None)
        awayTeam_def.append(None)
        continue

In [19]:
data = [homeTeam_attack, 
        homeTeam_mid, 
        homeTeam_def, 
        awayTeam_attack,
        awayTeam_mid, 
        awayTeam_def]

feature_cols = ['homeTeam_attack', 
                'homeTeam_mid', 
                'homeTeam_def', 
                'awayTeam_attack', 
                'awayTeam_mid',
                'awayTeam_def']

home_cols = ['homeTeam_attack', 
                'homeTeam_mid', 
                'homeTeam_def', 
                'awayTeam_attack', 
                'awayTeam_mid',
                'awayTeam_def']

away_cols = ['awayTeam_attack', 
             'awayTeam_mid',
             'awayTeam_def',
             'homeTeam_attack', 
             'homeTeam_mid', 
             'homeTeam_def']

rename_cols = ['A_Team_attack', 
               'A_Team_mid',
               'A_Team_def',
               'B_Team_attack', 
               'B_Team_mid', 
               'B_Team_def']
             

team_ratings_df = pd.DataFrame(data, index=feature_cols)

In [20]:
team_ratings_df = team_ratings_df.transpose()
team_ratings_df['HomeGoals'] = league_results_df['HomeGoals']
team_ratings_df['AwayGoals'] = league_results_df['AwayGoals']
team_ratings_df['result'] = league_results_df['Result']

In [21]:
team_ratings_df.dropna(inplace=True)

In [22]:
team_ratings_df.head()

,homeTeam_attack,homeTeam_mid,homeTeam_def,awayTeam_attack,awayTeam_mid,awayTeam_def,HomeGoals,AwayGoals,result
0,85.0,83.0,83.0,80.0,78.0,78.0,4,3,HomeWin
1,76.0,75.0,75.0,89.0,83.0,85.0,0,2,AwayWin
2,86.0,85.0,84.0,75.0,77.0,75.0,2,3,AwayWin
3,78.0,76.0,79.0,75.0,73.0,74.0,0,3,AwayWin
4,81.0,81.0,82.0,78.0,79.0,77.0,1,0,HomeWin


In [23]:
X = team_ratings_df[feature_cols]

In [24]:
X.tail()

,homeTeam_attack,homeTeam_mid,homeTeam_def,awayTeam_attack,awayTeam_mid,awayTeam_def
1441,81.0,81.0,80.0,88.0,86.0,88.0
1442,80.0,80.0,81.0,79.0,80.0,79.0
1443,86.0,85.0,83.0,78.0,78.0,78.0
1444,86.0,85.0,86.0,80.0,80.0,80.0
1445,79.0,80.0,80.0,79.0,77.0,77.0


In [25]:
print(X.shape)
X.iloc[[0,1370]]

(1446, 6)


,homeTeam_attack,homeTeam_mid,homeTeam_def,awayTeam_attack,awayTeam_mid,awayTeam_def
0,85.0,83.0,83.0,80.0,78.0,78.0
1370,80.0,78.0,78.0,81.0,81.0,81.0


In [26]:
result_mapping_dict = {'HomeWin': 2, 'Draw': 1, 'AwayWin': 0}
y = pd.DataFrame(league_results_df['Result']).reset_index(drop=True).replace(result_mapping_dict)
y.head()

,Result
0,2
1,0
2,0
3,0
4,2


In [27]:
print(y.shape)


(1446, 1)


# Machine Learning

In [28]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=3)

/Users/Michael/anaconda3/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [29]:
# default split is 75% for training and 25% for testing
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1084, 6)
(1084, 1)
(362, 6)
(362, 1)


In [30]:
# import model
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn import tree

clf = tree.DecisionTreeClassifier(random_state=1)
clf = clf.fit(X_train, y_train)

# instantiate
linreg = LinearRegression()
logreg = LogisticRegression(solver='newton-cg', multi_class='multinomial')
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)
logreg.fit(X_train, y_train)
clf.fit(X_train, y_train)

/Users/Michael/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/Michael/anaconda3/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:912: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [31]:
# # print the intercept and coefficients
# print(linreg.intercept_)
# print(linreg.coef_)

# print(logreg.intercept_)
# print(logreg.coef_)

In [32]:
# pair the feature names with the coefficients
# list(zip(feature_cols, logreg.coef_))

In [114]:
# make predictions on the testing set
linreg_y_pred = linreg.predict(X_test)
logreg_y_pred = logreg.predict(X_test)
clf_y_pred = clf.predict(X_test)

In [124]:
def get_numbers_from_array(array):
    nums = []
    for n in array:
        try:
            nums.append(n[0])
        except:
            nums.append(n)
    return nums

In [125]:
linreg_y_pred = get_numbers_from_array(linreg_y_pred)
logreg_y_pred = get_numbers_from_array(logreg_y_pred)
clf_y_pred = get_numbers_from_array(clf_y_pred)

In [126]:
# import seaborn

In [127]:
# seaborn.distplot(y_pred)
# plt.show()

In [128]:
# seaborn.distplot(test_model_df['real_results'])
# plt.show()

In [129]:
# seaborn.distplot(test_model_df['normalized predictions'])
# plt.show()

In [130]:
ml_data = [linreg_y_pred, logreg_y_pred, clf_y_pred]

In [133]:
test_model_df = pd.DataFrame(ml_data, 
                             index=['lin Reg', 'log Reg', 'clf']
                            ).transpose()

In [207]:
test_model_df['actual results'] = list(y_test['Result'])
test_model_df.head()

,lin Reg,log Reg,clf,actual results,poisson
0,1,2.0,2.0,2,2.0
1,1,2.0,2.0,0,2.0
2,2,2.0,2.0,2,2.0
3,1,2.0,2.0,2,2.0
4,2,2.0,2.0,2,2.0


In [136]:
def normalize_test_model(y):
    if y > (4/3):
        return 2
    elif y < (2/3):
        return 0
    else:
        return 1

In [157]:
test_model_df['lin Reg'] = test_model_df['lin Reg'].apply(normalize_test_model)
test_model_df.head()

,lin Reg,log Reg,clf,actual results
0,1,2.0,2.0,2
1,1,2.0,2.0,0
2,2,2.0,2.0,2
3,1,2.0,2.0,2
4,2,2.0,2.0,2


# Poisson

In [47]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

home_team_df = team_ratings_df[home_cols].copy()
home_team_df.columns = rename_cols
away_team_df = team_ratings_df[away_cols].copy()
away_team_df.columns = rename_cols

X = pd.concat([home_team_df, away_team_df]).reset_index(drop=True)

y = pd.DataFrame(pd.concat([team_ratings_df['HomeGoals'], team_ratings_df['AwayGoals']]).reset_index(drop=True), columns=['goals'])

goal_model_data = X.join(y)

goal_model_data.head()

/Users/Michael/anaconda3/lib/python3.5/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


,A_Team_attack,A_Team_mid,A_Team_def,B_Team_attack,B_Team_mid,B_Team_def,goals
0,85.0,83.0,83.0,80.0,78.0,78.0,4
1,76.0,75.0,75.0,89.0,83.0,85.0,0
2,86.0,85.0,84.0,75.0,77.0,75.0,2
3,78.0,76.0,79.0,75.0,73.0,74.0,0
4,81.0,81.0,82.0,78.0,79.0,77.0,1


In [48]:
poisson_model = smf.glm(formula="goals ~ A_Team_attack + A_Team_mid + A_Team_def + B_Team_attack + B_Team_mid + B_Team_def", 
                        data=goal_model_data, 
                        family=sm.families.Poisson()).fit()

poisson_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                  goals   No. Observations:                 2892
Model:                            GLM   Df Residuals:                     2885
Model Family:                 Poisson   Df Model:                            6
Link Function:                    log   Scale:                             1.0
Method:                          IRLS   Log-Likelihood:                -4221.6
Date:                Wed, 13 Jun 2018   Deviance:                       3404.4
Time:                        12:02:57   Pearson chi2:                 3.01e+03
No. Iterations:                     5                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        -1.6821      0.507     -3.317      0.001      -2.676      -0.688
A_Team_attack     0.0434      0.011      3.934      0.000       0.022       0.065
A_Team_mid        0.0020      0.012      0.170      0.865      -0.021       0.025
A_Team_def        0.0259      0.012      2.113      0.035       0.002       0.050
B_Team_attack    -0.0129      0.011     -1.149      0.250      -0.035       0.009
B_Team_mid       -0.0359      0.012     -2.903      0.004      -0.060      -0.012
B_Team_def        0.0020      0.013      0.155      0.877      -0.023       0.027
=================================================================================
"""

In [208]:
def return_poisson_predictions(a_team, b_team, type='Club'):
    game = simulate_match(poisson_model, a_team, b_team, type=type)
    a_team_odds, b_team_odds, draw_odds, a_goals, b_goals = show_game_summary(game)
    print(a_team_odds)
    print(b_team_odds)
    print(draw_odds)
    if a_team_odds > b_team_odds:
        if a_team_odds > draw_odds:
            return 2
        if a_team_odds < draw_odds:
            return 1
    if a_team_odds < b_team_odds:
        if b_team_odds > draw_odds:
            return 0
        if b_team_odds < draw_odds:
            return 1

In [209]:
y_test

,Result
616,2
979,0
977,2
972,2
83,2
1381,1
1101,0
50,1
751,0
1414,2


In [210]:
poisson_X_test_df = X_test.reset_index().join(league_results_df.reset_index(), on='index', how='left', lsuffix='L', rsuffix='R')
poisson_X_test_df

,indexL,homeTeam_attack,homeTeam_mid,homeTeam_def,awayTeam_attack,awayTeam_mid,awayTeam_def,indexR,HomeTeam,AwayTeam,HomeGoals,AwayGoals,Result,Goal diff
0,616,78.0,78.0,77.0,76.0,76.0,76.0,616,Eintracht Frankfurt,1. FSV Mainz 05,3,0,HomeWin,3
1,979,78.0,76.0,75.0,76.0,76.0,76.0,979,Atalanta,Sampdoria,1,2,AwayWin,-1
2,977,81.0,80.0,80.0,73.0,73.0,73.0,977,Lazio,Benevento Calcio,6,2,HomeWin,4
3,972,77.0,76.0,76.0,76.0,76.0,76.0,972,Chievo Verona,Sampdoria,2,1,HomeWin,1
4,83,89.0,83.0,85.0,75.0,77.0,75.0,83,Manchester City,Burnley,3,0,HomeWin,3
5,1381,81.0,81.0,81.0,81.0,81.0,80.0,1381,Sevilla FC,Villarreal CF,2,2,Draw,0
6,1101,77.0,76.0,75.0,81.0,81.0,80.0,1101,Deportivo Alavés,Villarreal CF,0,3,AwayWin,-3
7,50,75.0,77.0,75.0,75.0,73.0,74.0,50,Burnley,Huddersfield Town,0,0,Draw,0
8,751,81.0,82.0,83.0,83.0,81.0,82.0,751,Milan,Roma,0,2,AwayWin,-2
9,1414,81.0,81.0,81.0,80.0,80.0,80.0,1414,Sevilla FC,Real Sociedad,1,0,HomeWin,1


In [211]:
poisson_test_results = []

for index, row in poisson_X_test_df[['HomeTeam', 'AwayTeam']].iterrows():
    a_team = row['HomeTeam']
    b_team = row['AwayTeam']
    poisson_test_results.append(return_poisson_predictions(a_team, b_team))


poisson_test_results

0.4258089712012538
0.30227819344303186
0.27191104469824745
0.3889333578082771
0.33705833001305696
0.2740069295950445
0.6380368034706905
0.14796139545766795
0.2139700684698593
0.3791721534397032
0.3462220104409309
0.27460452822924336
0.7736701852608249
0.08289776904406637
0.14303093640852552
0.37729416678049216
0.3624877790483635
0.26021514352599145
0.22102489311687554
0.5326295972679966
0.246336703914647
0.40906443094636863
0.30968251943023983
0.28125201332764066
0.35882450934063914
0.39136267574733663
0.2498073725037964
0.40704442831636417
0.3341539632577996
0.25879827064331984
0.6048726036513787
0.17678012077535854
0.2183154576709066
0.3376409486679263
0.38782303712170374
0.27453467402925613
0.16838792942895187
0.629678746705655
0.20186295823994918
0.3735553624808058
0.3514151980190736
0.2750281728283765
0.5074035798697338
0.24422771022748035
0.24836086363800614
0.434876686287669
0.29222180427837685
0.27289972702686205
0.32723181450002187
0.39138991283947155
0.2813773216940737
0.7380

0.7044975434468952
0.11414971140364288
0.1812485071550637
0.4742303549601521
0.2673797944720031
0.2583855028559513
0.1345223808270742
0.6842895328561002
0.1810440375743901
0.5286495900358182
0.22365823647380542
0.24768395037069554
0.3793843576926402
0.3435047422504457
0.27710975543763616
0.2915602192949184
0.4348567733874855
0.2735812837882154
0.483407961848139
0.2621794586166722
0.254407102962287
0.7257412220127257
0.11086159180433058
0.1631436890001412
0.5697866456652895
0.20964628147489245
0.22053353890496485
0.5115955786285572
0.24294081568206533
0.24545442772053486
0.35882450934063914
0.39136267574733663
0.2498073725037964
0.2717187829248099
0.4538342279530331
0.27444517120809897
0.7830943211302509
0.07502502631984334
0.14153021476528635
0.618790557588405
0.16010071709083182
0.22108393494857229
0.4950212957213667
0.2539132627589542
0.2510587529203305
0.2351120578338429
0.516847690351629
0.24803220256304984
0.4735052304883451
0.26376675127064647
0.26272453111859395
0.49278539483351

0.43090094774360593
0.3282853194928947
0.2408036484156227
0.2678611270283079
0.4831154306018637
0.24901618031437858
0.21339788021958284
0.5453816915125833
0.24120905594198433
0.44048688230922883
0.2910270712981967
0.26848376589559275
0.3913899128394715
0.3272318145000219
0.2813773216940737
0.4003997850855469
0.3311004639636429
0.26849782871732236
0.6020863173804168
0.18034984380476937
0.21752996012296308
0.16739656301587413
0.6424111305292548
0.19005931723172978
0.5962322253879792
0.19241226577102677
0.21130472178733317
0.6171274347512398
0.15415748223114462
0.22869847024614495
0.3077916331251273
0.4162080539598023
0.275998920347105
0.5783584475502347
0.18380354697892323
0.23782549349254523
0.23753824043999638
0.4954252144517379
0.2670335113811345
0.2753051634446807
0.4547196793402825
0.2699728940819766
0.12345977641078207
0.6931159839983341
0.18331572491602147
0.27332633301949244
0.4743371186391504
0.2523305946859516
0.2222047160628858
0.5415186037859709
0.2362617856608623
0.151732334

[2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 2,
 0,
 0,
 0,
 2,
 2,
 2,
 0,
 2,


In [205]:
test_model_df['poisson'] = poisson_test_results
test_model_df

,lin Reg,log Reg,clf,actual results,poisson
0,1,2.0,2.0,2,2.0
1,1,2.0,2.0,0,2.0
2,2,2.0,2.0,2,2.0
3,1,2.0,2.0,2,2.0
4,2,2.0,2.0,2,2.0
5,1,2.0,2.0,1,2.0
6,1,0.0,2.0,0,0.0
7,1,2.0,2.0,1,2.0
8,1,2.0,2.0,0,0.0
9,1,2.0,2.0,2,2.0


In [206]:
def compare_accuracy(df):
    count1 = 0
    count2 = 0
    count3 = 0
    count4 = 0

    for index, row in df.iterrows():
        if row['lin Reg'] == row['actual results']:
            count1 += 1
        else:
            continue
        if row['log Reg'] == row['actual results']:
            count2 += 1
        else:
            continue
        if row['clf'] == row['actual results']:
            count3 += 1
        else:
            continue
        if row['poisson'] == row['actual results']:
            count4 += 1
        else:
            continue


    print('Linear Reg Model: {}% accurate'.format(round(count1 / len(test_model_df), 2)*100))
    print('Logistic Reg Model: {}% accurate'.format(round(count2 / len(test_model_df), 2)*100))
    print('Classification Model: {}% accurate'.format(round(count3 / len(test_model_df), 2)*100))
    print('Poisson Model: {}% accurate'.format(round(count4 / len(test_model_df), 2)*100))
          
compare_accuracy(test_model_df)

Linear Reg Model: 44.0% accurate
Logistic Reg Model: 26.0% accurate
Classification Model: 19.0% accurate
Poisson Model: 19.0% accurate


# Running Game Simulations

In [51]:
def simulate_goals(game, a_team='A Team', b_team='B Team', verbose=False):
    a_goals = 0
    b_goals = 0
    outcome = 0

    random_event = np.random.random()
    counter = 0


    for index, row in pd.DataFrame(data=game).iterrows():
        for x in range(len(row)):
            counter += row[x]
            
            a_goals = index
            b_goals = x
            
            if counter > random_event:
                break
        if counter > random_event:
            break
    
    if verbose == True:
        print('Random float: {}'.format(random_event))
        print('{} goals: {}'.format(a_team, str(a_goals)))
        print('{} goals: {}\n'.format(b_team, str(b_goals)))
    
    return a_goals, b_goals


def show_game_summary(game, a_team='A Team', b_team='B Team', verbose=False):
    a_team_odds = np.sum(np.tril(game, -1))
    b_team_odds = np.sum(np.triu(game, 1))
    draw_odds = np.sum(np.diag(game))
    
    if verbose == True:
        print('{} chance of Win: {}%'.format(a_team, round(a_team_odds*100, 2)))
        print('{} chance of Win: {}%'.format(b_team, round(b_team_odds*100, 2)))
        print('Odds of a draw: {}%'.format(round(draw_odds*100, 2)))
    
    a_goals, b_goals = simulate_goals(game, a_team, b_team, verbose=verbose)
    
    return a_team_odds, b_team_odds, draw_odds, a_goals, b_goals


def simulate_match(foot_model, a_team, b_team, type='Club', max_goals=9, verbose=False):
    a_team_stats = get_team_stats(a_team, type=type)
    b_team_stats = get_team_stats(b_team, type=type)
    
    data1 = {'A_Team_attack': a_team_stats[0],
              'A_Team_mid': a_team_stats[1],
              'A_Team_def': a_team_stats[2],
              'B_Team_attack': b_team_stats[0], 
              'B_Team_mid': b_team_stats[1], 
              'B_Team_def': b_team_stats[2]}
    
    data2 = {'A_Team_attack': b_team_stats[0],
              'A_Team_mid': b_team_stats[1],
              'A_Team_def': b_team_stats[2],
              'B_Team_attack': a_team_stats[0], 
              'B_Team_mid': a_team_stats[1], 
              'B_Team_def': a_team_stats[2]}
        
    a_goals_avg = foot_model.predict(pd.DataFrame(data=data1, index=[1])).values[0]
    b_goals_avg = foot_model.predict(pd.DataFrame(data=data2, index=[1])).values[0]
    
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in [a_goals_avg, b_goals_avg]]
    game = (np.outer(np.array(team_pred[0]), np.array(team_pred[1])))
    show_game_summary(game, a_team, b_team, verbose=verbose)
    return game


In [100]:
game = simulate_match(poisson_model, 'Iran', 'Spain', type='Nationality')
show_game_summary(game, 'Iran', 'Spain', verbose=True)

Iran chance of Win: 2.96%
Spain chance of Win: 88.98%
Odds of a draw: 7.85%
Random float: 0.44281062990106224
Iran goals: 0
Spain goals: 4



(0.029581164489326576, 0.8897511047391475, 0.07849130091609607, 0, 4)

In [53]:
simulate_goals(game)

(1, 2)

## Getting the world cup squad Stats

In [54]:
get_team_stats('Belgium', type='Nationality', return_df=True)

Overall
Nationality Preferred Positions         
Belgium     Attack                  86.0
            Defense                 86.0
            Midfield                84.0

In [55]:
get_team_stats('Sweden', type='Nationality', return_df=True)

Overall
Nationality Preferred Positions         
Sweden      Attack                  80.0
            Defense                 78.0
            Midfield                80.0

In [56]:
schedule_df = pd.read_csv('resources/worldcup_match_schedule.csv')
schedule_df.head()

,a_team,b_team
0,Russia,Saudi Arabia
1,Egypt,Uruguay
2,Portugal,Spain
3,Morocco,Iran
4,France,Australia


In [57]:
def get_bulk_team_stats(schedule):


    a_team_attack = []
    a_team_mid = []
    a_team_def = []
    b_team_attack = []
    b_team_mid = []
    b_team_def = []
    data = []


    for index, row in schedule.iterrows():
        try:
            a_team_stats = get_team_stats(row['a_team'], type='Nationality')
            a_attack = a_team_stats[0][0]
            a_midfield = a_team_stats[0][1]
            a_defense = a_team_stats[0][2]

            b_team_stats = get_team_stats(row['b_team'], type='Nationality')
            b_attack = b_team_stats[0][0]
            b_midfield = b_team_stats[0][1]
            b_defense = b_team_stats[0][2]

            if len(a_team_stats[0]) != 3 or len(b_team_stats[0]) != 3:
                a_team_attack.append(None)
                a_team_mid.append(None)
                a_team_def.append(None)
                b_team_attack.append(None)
                b_team_mid.append(None)
                b_team_def.append(None)
            else:            
                a_team_attack.append(a_attack)
                a_team_mid.append(a_midfield)
                a_team_def.append(a_defense)
                b_team_attack.append(b_attack)
                b_team_mid.append(b_midfield)
                b_team_def.append(b_defense)


        except IndexError:
            a_team_attack.append(None)
            a_team_mid.append(None)
            a_team_def.append(None)
            b_team_attack.append(None)
            b_team_mid.append(None)
            b_team_def.append(None)
            continue

    data = [a_team_attack, 
            a_team_mid, 
            a_team_def, 
            b_team_attack,
            b_team_mid, 
            b_team_def]

    rename_cols = ['A_Team_attack', 
                   'A_Team_mid',
                   'A_Team_def',
                   'B_Team_attack', 
                   'B_Team_mid', 
                   'B_Team_def']


    return pd.DataFrame(data, index=rename_cols).transpose()

In [58]:
# mask = overall_score_df['Nationality'] == 'Serbia'
# groupby_cols = ['Nationality', 'Preferred Positions']
# player_overall_df[mask].head(13).groupby(groupby_cols).mean().round(0)

In [59]:
def get_game_summary(df):

    a_team_odds = []
    b_team_odds = []
    draw_odds = []
    a_goals_count = []
    b_goals_count = []

    for index, row in schedule_df.iterrows():
        a_team = row[0]
        b_team = row[1]

        game = simulate_match(poisson_model, a_team, b_team, type='Nationality')
        a_team_probability, b_team_probability, draw_probability, a_goals, b_goals  = show_game_summary(game)

        a_team_odds.append(a_team_probability)
        b_team_odds.append(b_team_probability)
        draw_odds.append(draw_probability)
        a_goals_count.append(a_goals)
        b_goals_count.append(b_goals)
        
    
    df['a_team_chances'] = pd.Series(a_team_odds)
    df['b_team_chances'] = pd.Series(b_team_odds)
    df['draw_chances'] = pd.Series(draw_odds)
    df['a_goals'] = pd.Series(a_goals_count)
    df['b_goals'] = pd.Series(b_goals_count)
    
    return df

In [101]:
def calculate_stage_winner(df):
    a_team_points = []
    b_team_points = []

    for index, row in group_stage_results_df.iterrows():
        win = 3
        loss = 0
        draw = 1
        if row['a_goals'] == row['b_goals']:
            a_team_points.append(draw)
            b_team_points.append(draw)
        elif row['a_goals'] > row['b_goals']:
            a_team_points.append(win)
            b_team_points.append(loss)
        elif row['a_goals'] < row['b_goals']:
            a_team_points.append(loss)
            b_team_points.append(win)


        df['a_points'] = pd.Series(a_team_points)
        df['b_points'] = pd.Series(b_team_points)
        
    return df

group_stage_results_df = get_game_summary(schedule_df)
group_stage_results_df = calculate_stage_winner(group_stage_results_df)
group_stage_results_df

,a_team,b_team,a_team_chances,b_team_chances,draw_chances,a_goals,b_goals,a_points,b_points
0,Russia,Saudi Arabia,0.554290,0.202677,0.243023,2,3,0,3
1,Egypt,Uruguay,0.128553,0.675456,0.195929,0,1,0,3
2,Portugal,Spain,0.311893,0.449925,0.238170,0,3,0,3
3,Morocco,Iran,0.632917,0.147130,0.219929,2,0,3,0
4,France,Australia,0.783094,0.075025,0.141530,3,0,3,0
5,Peru,Denmark,0.210984,0.542639,0.246369,0,2,0,3
6,Argentina,Iceland,0.823846,0.058962,0.116335,0,1,0,3
7,Croatia,Nigeria,0.597850,0.175881,0.226247,1,0,3,0
8,Brazil,Switzerland,0.611717,0.175833,0.212408,4,1,3,0
9,Costa Rica,Serbia,0.214069,0.543399,0.242522,1,3,0,3


In [102]:
cols = {'a_team': 'Team', 'a_goals': 'Goals', 'a_points': 'Points',
        'b_team': 'Team', 'b_goals': 'Goals', 'b_points': 'Points'}

a_team_group_results_df = group_stage_results_df[['a_team', 'a_goals', 'a_points']].rename(columns=cols)
b_team_group_results_df = group_stage_results_df[['b_team', 'b_goals', 'b_points']].rename(columns=cols)

scoreboard_df = pd.concat([a_team_group_results_df, b_team_group_results_df]).reset_index(drop=True)
scoreboard_df = scoreboard_df.merge(pd.read_csv('resources/worldcup_country_groups.csv'), on='Team', how='left')
scoreboard_df = scoreboard_df.groupby(['Group', 'Team']).sum().reset_index()
scoreboard_df = scoreboard_df.sort_values(['Group', 'Points', 'Goals'], ascending=[True, False, False]).reset_index(drop=True)
scoreboard_df

,Group,Team,Goals,Points
0,A,Russia,6,6
1,A,Saudi Arabia,5,6
2,A,Uruguay,4,6
3,A,Egypt,0,0
4,B,Spain,6,9
5,B,Portugal,7,6
6,B,Morocco,2,3
7,B,Iran,1,0
8,C,France,9,9
9,C,Australia,4,6


In [62]:
round_of_16_games = [(0, 5), (8, 13),
                     (16, 21), (24, 29),
                     (4, 1), (12, 9),
                     (20, 17), (28, 25)]

round_of_8_games = [(0, 1), (2, 3),
                     (4, 5), (6, 7)]

round_of_4_games = [(0, 1), (2, 3)]

round_of_2_games = [(0, 1)]

def get_team_penalties(team):
    mask = players_df['Nationality'] == team
    return pd.to_numeric(players_df[mask]['Penalties'].head(), errors='coerce').mean()

def simulate_penalties(a_team, b_team):
    a_pen = get_team_penalties(a_team)
    b_pen = get_team_penalties(b_team)
    
    if a_pen == b_pen:
        winner = a_team
    elif a_pen > b_pen:
        winner = a_team
    elif a_pen < b_pen:
        winner = b_team
    
    return winner

def get_knockout_winner(round_results_df):
    winners = []
    
    for index, row in round_results_df.iterrows():
        if row['a_goals'] == row['b_goals']:
            winner = simulate_penalties(row[0], row[1])
        elif row['a_goals'] > row['b_goals']:
            winner = row[0]
        elif row['a_goals'] < row['b_goals']:
            winner = row[1]
            
        winners.append(winner)
        
    round_results_df['Winner'] = pd.Series(winners)
    
    return round_results_df
    

def simulate_knockout_stage(df, schedule_tuples):
    a_team_lists = []
    b_team_lists = []

    a_team_odds = []
    b_team_odds = []
    draw_odds = []
    a_goals_count = []
    b_goals_count = []
    
    
    for match in schedule_tuples:
        a_team = df.iloc[match[0]]['Team']
        b_team = df.iloc[match[1]]['Team']
        a_team_lists.append(a_team)
        b_team_lists.append(b_team)

        game = simulate_match(poisson_model, a_team, b_team, type="Nationality")
        a_team_probability, b_team_probability, draw_probability, a_goals, b_goals  = show_game_summary(game)

        a_team_odds.append(a_team_probability)
        b_team_odds.append(b_team_probability)
        draw_odds.append(draw_probability)
        a_goals_count.append(a_goals)
        b_goals_count.append(b_goals)

    round_results_df = pd.DataFrame([a_team_lists, b_team_lists]).transpose()
    round_results_df = get_game_summary(round_results_df)
    round_results_df = get_knockout_winner(round_results_df)
    
    df = pd.DataFrame(round_results_df['Winner']).rename(columns={'Winner': 'Team'})    
    
    return df



In [63]:
def simulate_World_Cup(n_simulations=None, verbose=False):
    world_cup_winners = []
    n = 0
    while n < n_simulations:
        try:
            group_stage_results_df = get_game_summary(schedule_df)
            group_stage_results_df = calculate_stage_winner(group_stage_results_df)

            cols = {'a_team': 'Team', 'a_goals': 'Goals', 'a_points': 'Points',
                    'b_team': 'Team', 'b_goals': 'Goals', 'b_points': 'Points'}

            a_team_group_results_df = group_stage_results_df[['a_team', 'a_goals', 'a_points']].rename(columns=cols)
            b_team_group_results_df = group_stage_results_df[['b_team', 'b_goals', 'b_points']].rename(columns=cols)

            scoreboard_df = pd.concat([a_team_group_results_df, b_team_group_results_df]).reset_index(drop=True)
            scoreboard_df = scoreboard_df.merge(pd.read_csv('resources/worldcup_country_groups.csv'), on='Team', how='left')
            scoreboard_df = scoreboard_df.groupby(['Group', 'Team']).sum().reset_index()
            scoreboard_df = scoreboard_df.sort_values(['Group', 'Points', 'Goals'], ascending=[True, False, False])

            df_8 = simulate_knockout_stage(scoreboard_df, round_of_16_games)
            df_4 = simulate_knockout_stage(df_8, round_of_8_games)
            df_2 = simulate_knockout_stage(df_4, round_of_4_games)
            world_cup_winner = simulate_knockout_stage(df_2, round_of_2_games).iloc[0].item()

            world_cup_winners.append(world_cup_winner)
            
            if verbose == True:
                print(group_stage_results_df)
                print(scoreboard_df)
                print(df_8)
                print(df_4)
                print(df_2)
                print(pd.Series(world_cup_winners).value_counts())
                
            print('The winner of World Cup 2018 simulation {}/{} is {}'.format(n+1, n_simulations, world_cup_winner))
            

            n += 1
        except:
            print('Exception occured')
            continue
        
    return pd.Series(world_cup_winners).value_counts()
        


In [103]:
world_cup_results = simulate_World_Cup(1, verbose=True)

            a_team          b_team  a_team_chances  b_team_chances  \
0           Russia    Saudi Arabia        0.554290        0.202677   
1            Egypt         Uruguay        0.128553        0.675456   
2         Portugal           Spain        0.311893        0.449925   
3          Morocco            Iran        0.632917        0.147130   
4           France       Australia        0.783094        0.075025   
5             Peru         Denmark        0.210984        0.542639   
6        Argentina         Iceland        0.823846        0.058962   
7          Croatia         Nigeria        0.597850        0.175881   
8           Brazil     Switzerland        0.611717        0.175833   
9       Costa Rica          Serbia        0.214069        0.543399   
10         Germany          Mexico        0.657188        0.150520   
11          Sweden  Korea Republic        0.518825        0.238300   
12         Belgium          Panama        0.923147        0.015909   
13         Tunisia  

In [65]:
world_cup_results

NameError: name 'world_cup_results' is not defined